# Importing the necessary packages

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
# System packages
import os
import csv
from IPython.display import clear_output
import time
from dataclasses import dataclass
import copy

# Science Packages
import numpy as np
import scipy as sc
from scipy import io
import matplotlib.pyplot as plt

# Data Packages
import h5py
import json

# CAD Packages
import gdstk
import lib.GDSTKUtils as utils
import lib.dither_utils as dit
import lib.GDSTKMasks as mask_util

# Import Parameters
from lib.mask_dict import *


In [4]:
# default size unit is microns, so define handy shortcuts for other units
um = 1
mm = 1e3
cm = 1e4

In [5]:
@dataclass
class dither_obj(object):
    x: np.matrix
    y: np.matrix
    block_size: np.matrix
        

In [6]:
def invert_coords(x_arr, y_arr):
    tmp_x = y_arr
    tmp_y = x_arr
    return tmp_x, tmp_y

def convert_coords(dims, x_arr, y_arr):
    if type(dims) == float:
        tmp_y = dims - y_arr
        tmp_x = x_arr
    elif type(dims) == list:
        tmp_y = dims[0] - y_arr
        tmp_x = x_arr
    return tmp_x, tmp_y

def shift_to_center(dims, x_arr, y_arr):
    if type(dims) == float:
        tmp_x = x_arr - (dims/ 2)
        tmp_y = y_arr - (dims/ 2)
    elif type(dims) == list:
        tmp_x = x_arr - (dims[0]/ 2)
        tmp_y = y_arr - (dims[1]/ 2)
    return tmp_x, tmp_y

def shift_to_zero(dims, x_arr, y_arr):
    if type(dims) == float:
        tmp_x = x_arr + (dims/ 2)
        tmp_y = y_arr + (dims/ 2)
    elif type(dims) == list:
        tmp_x = x_arr + (dims[0]/ 2)
        tmp_y = y_arr + (dims[1]/ 2)
    return tmp_x, tmp_y

def merge_centers(led_real_pos, pat_center, dims, invert = True, index = 0):
    if invert == True:
        center_led = [(led_real_pos['x'][index, dims[1]-1] + led_real_pos['x'][index,0])/2, (led_real_pos['y'][0,index] + led_real_pos['y'][dims[0]-1, index])/2]
    elif invert == False:
        center_led = [(led_real_pos['x'][index, dims[1] -1] + led_real_pos['x'][index,0])/2, (led_real_pos['y'][dims[0]-1, index] + led_real_pos['y'][0, index])/2]
    shift = [pat_center[0] - center_led[0] , pat_center[1] - center_led[1]] 
    return shift

## Reading in data
### Matlab files

In [7]:
# Loading the Matrix
mat_path = os.path.join("Data", "Positions", "221129 Masks", "mnistspread_bleed_221025205444_cellarray_flipboth.mat")
mat = io.loadmat(mat_path)

# Reading cell data
net0 = mat['net0cell']
net1 = mat['net1cell']
net2 = mat['net2cell']

### h5 files

In [8]:
# File locations
# LED_input_file = os.path.join("Data", "Positions", "positions", "LED_positions_input.h5")
LED_input_file = 'Data/Positions/221129 Masks/LED_positions_input_test.h5'
LED_dev1_file = os.path.join("Data", "Positions", "positions", "LED_positions_device1.h5")
LED_dev2_file = os.path.join("Data", "Positions", "positions", "LED_positions_device2.h5")

PD_dev1_file = os.path.join("Data", "Positions", "positions", "PD_positions_device_1_flipboth.h5")
PD_dev2_file = os.path.join("Data", "Positions", "positions", "PD_positions_device_2_flipboth.h5")
# PD_dev2_file = 'Data/Positions/221129 Masks/PD_positions_device_2_test.h5'

PD_output_file = os.path.join("Data", "Positions", "positions", "PD_positions_output_ideal_flipboth.h5")

# Loading the h5 file variables
LED_input_h5 = h5py.File(LED_input_file, "r")
LED_dev1_h5 = h5py.File(LED_dev1_file, "r")
LED_dev2_h5 = h5py.File(LED_dev2_file, "r")

PD_output_h5 = h5py.File(PD_output_file, "r")
PD_dev1_h5 = h5py.File(PD_dev1_file, "r")
PD_dev2_h5 = h5py.File(PD_dev2_file, "r")

## Obtaining the data and transposing
# Setting empty dictionaries
LED_input_cen, LED_dev1_cen, LED_dev2_cen, PD_dev1_cen, PD_dev2_cen, PD_output_cen = {}, {}, {}, {}, {}, {}

# Input
LED_input_cen['x'] = np.array(LED_input_h5.get('LED_cen_x'))*mm
LED_input_cen['y'] = np.array(LED_input_h5.get('LED_cen_y'))*mm
LED_input_cen['x'], LED_input_cen['y'] = np.transpose(LED_input_cen['x']), np.transpose(LED_input_cen['y'])
LED_input_cen['x'], LED_input_cen['y'] = utils.convert_coords([25*mm, 25*mm], LED_input_cen['x'], LED_input_cen['y'])


# LED1
LED_dev1_cen['x'] = np.array(LED_dev1_h5.get('LED_cen_x'))*mm
LED_dev1_cen['y'] = np.array(LED_dev1_h5.get('LED_cen_y'))*mm
LED_dev1_cen['x'], LED_dev1_cen['y'] = np.transpose(LED_dev1_cen['x']), np.transpose(LED_dev1_cen['y'])
LED_dev1_cen['x'], LED_dev1_cen['y'] = utils.convert_coords([25*mm, 25*mm], LED_dev1_cen['x'], LED_dev1_cen['y'])

# LED2
LED_dev2_cen['x'] = np.array(LED_dev2_h5.get('LED_cen_x'))*mm
LED_dev2_cen['y'] = np.array(LED_dev2_h5.get('LED_cen_y'))*mm
LED_dev2_cen['x'], LED_dev2_cen['y'] = np.transpose(LED_dev2_cen['x']), np.transpose(LED_dev2_cen['y'])
LED_dev2_cen['x'], LED_dev2_cen['y'] = utils.convert_coords([25*mm, 25*mm], LED_dev2_cen['x'], LED_dev2_cen['y'])

# PD1 
PD_dev1_cen['x'] = np.array(PD_dev1_h5.get('PD_cen_x'))*mm
PD_dev1_cen['y'] = np.array(PD_dev1_h5.get('PD_cen_y'))*mm
PD_dev1_cen['x'], PD_dev1_cen['y'] = np.transpose(PD_dev1_cen['x']), np.transpose(PD_dev1_cen['y'])
PD_dev1_cen['x'], PD_dev1_cen['y'] = utils.convert_coords([25*mm, 25*mm], PD_dev1_cen['x'], PD_dev1_cen['y'])


# PD2
PD_dev2_cen['x'] = np.array(PD_dev2_h5.get('PD_cen_x'))*mm
PD_dev2_cen['y'] = np.array(PD_dev2_h5.get('PD_cen_y'))*mm
PD_dev2_cen['x'], PD_dev2_cen['y'] = np.transpose(PD_dev2_cen['x']), np.transpose(PD_dev2_cen['y'])
PD_dev2_cen['x'], PD_dev2_cen['y'] = utils.convert_coords([25*mm, 25*mm], PD_dev2_cen['x'], PD_dev2_cen['y'])

# Output
PD_output_cen['x'] = np.array(PD_output_h5.get('PD_cen_x'))*mm
PD_output_cen['y'] = np.array(PD_output_h5.get('PD_cen_y'))*mm
PD_output_cen['x'], PD_output_cen['y'] = np.transpose(PD_output_cen['x']), np.transpose(PD_output_cen['y'])
PD_output_cen['x'], PD_output_cen['y'] = utils.convert_coords([25*mm, 25*mm], PD_output_cen['x'], PD_output_cen['y'])

In [9]:
json_file = os.path.join('Data', 'JSON', 'MNIST 215.json')

# Mask sheet

In [10]:
# File parameters 
sheet_height = 205*mm
sheet_width = 280*mm

# Initializing a GDS library
mask_array = gdstk.Library()
masks = mask_array.new_cell("Block Cell")

In [11]:
# Creating a meshgrid of center positions for all the masks
pos_x_arr = np.arange(20*mm, 280*mm, 35*mm)
pos_y_arr = np.arange(27.5*mm, 205*mm, 35*mm)
center_x, center_y = np.meshgrid(pos_x_arr, pos_y_arr)

In [12]:
# Constructing the list with the dictionaries
mask_list = [
    weight_mask_184_01, weight_mask_184_01, weight_mask_184_01,
    weight_mask_184_12, weight_mask_184_12, weight_mask_184_12,
    weight_mask_184_23, weight_mask_184_23, weight_mask_184_23,
    weight_mask_200_01, weight_mask_200_01, weight_mask_200_01,
    weight_mask_200_12, weight_mask_200_12, weight_mask_200_12,
    weight_mask_200_23, weight_mask_200_23, weight_mask_200_23,
    weight_mask_216_01, weight_mask_216_01, weight_mask_216_01,
    weight_mask_216_01, weight_mask_216_12, weight_mask_216_12,
    weight_mask_216_01, weight_mask_216_23, weight_mask_216_23,
    cb_mask_01, cb_mask_01, cb_mask_01,
    cb_mask_12, cb_mask_12, cb_mask_12,
    cb_mask_23, cb_mask_23, cb_mask_23,
    ph_mask_01, ph_mask_01, ph_mask_01, ph_mask_01,
    ph_mask_shifted_01, ph_mask_shifted_01, ph_mask_shifted_01, ph_mask_shifted_01,
    ph_mask_shifted_12, ph_mask_shifted_12, ph_mask_shifted_12, ph_mask_shifted_12,
]

In [13]:
index = 0
for x, y in zip(center_x.flatten(), center_y.flatten()):
    # masks = mask_util.generate_weight_mask_from_json(masks, "0", json_file, [x, y], net0, LED_input_cen, PD_dev1_cen)
    masks = mask_util.sel_mask(masks, mask_list[index], [x,y])
    index += 1

In [14]:
# Writing the mask array to the GDS file
mask_array.write_gds('221130 Mask.gds')

In [16]:
# File parameters 
sheet_height = 205*mm
sheet_width = 280*mm

# Initializing a GDS library
mask_array = gdstk.Library()
masks = mask_array.new_cell("Block Cell")

In [19]:
# masks = mask_util.sel_mask(masks, mask_list[0], [x,y])
masks = mask_util.chequerboard_array(masks, [0,0], 10, LED_input_cen, PD_dev1_cen, 'Test')

In [20]:
# Writing the mask array to the GDS file
mask_array.write_gds('test.gds')

In [17]:
# # Creating all the borders 
# for x, y in zip(center_x.flatten(), center_y.flatten()):
#     slide_border = mask_util.rect_border([x, y], [30.25*mm, 30.25*mm], 250*um)
#     masks.add(*slide_border)

## Storing data in JSON files
### Setting the different values for the different classes of masks

In [39]:
# Files for 185
mnist0_185 ={
    "name": "MNIST Input Layer",
    "title": "MNIST 0 (185um)",
    "magnification": 10,
    "dither_size": 185*um,
}

mnist1_185 ={
    "name": "MNIST First Layer",
    "title": "MNIST 1 (185um)",
    "magnification": 12.5,
    "dither_size": 185*um,
}

mnist2_185 ={
    "name": "MNIST Second Layer",
    "title": "MNIST 2 (185um)",
    "magnification": 12.5,
    "dither_size": 185*um,
}

master_dict_185 = {
    '0': mnist0_185,
    '1': mnist1_185,
    '2': mnist2_185,
}

with open("MNIST 185.json", "w") as outfile:
    json.dump(master_dict_185, outfile)

In [40]:
# Files for 200
mnist0_200 ={
    "name": "MNIST Input Layer",
    "title": "MNIST 0 (200um)",
    "magnification": 10,
    "dither_size": 200*um,
}

mnist1_200 ={
    "name": "MNIST First Layer",
    "title": "MNIST 1 (200um)",
    "magnification": 12.5,
    "dither_size": 200*um,
}

mnist2_200 ={
    "name": "MNIST Second Layer",
    "title": "MNIST 2 (200um)",
    "magnification": 12.5,
    "dither_size": 200*um,
}

master_dict_200 = {
    '0': mnist0_200,
    '1': mnist1_200,
    '2': mnist2_200,
}

with open("MNIST 200.json", "w") as outfile:
    json.dump(master_dict_200, outfile)

In [41]:
# Files for 225
mnist0_215 ={
    "name": "MNIST Input Layer",
    "title": "MNIST 0 (215um)",
    "magnification": 10,
    "dither_size": 215*um,
}

mnist1_215 ={
    "name": "MNIST First Layer",
    "title": "MNIST 1 (215um)",
    "magnification": 12.5,
    "dither_size": 215*um,
}

mnist2_215 ={
    "name": "MNIST Second Layer",
    "title": "MNIST 2 (215um)",
    "magnification": 12.5,
    "dither_size": 215*um,
}

master_dict_215 = {
    '0': mnist0_215,
    '1': mnist1_215,
    '2': mnist2_215,
}

with open("MNIST 215.json", "w") as outfile:
    json.dump(master_dict_215, outfile)

# Individual Mask

### Chequerboard Mask

In [184]:
sheet_height = 205*mm
sheet_width = 280*mm

# Initializing a GDSPY library
gds_array = gdstk.Library()

# GDSPY initialization stuff
gds_mask = gds_array.new_cell("Block Cell")

In [185]:
led_real_pos = copy.deepcopy(LED_dev2_cen)
pd_real_pos = copy.deepcopy(PD_output_cen)
pd_real_pos['x'], pd_real_pos['y'] = utils.shift_to_center([25*mm, 25*mm], pd_real_pos['x'], pd_real_pos['y'])

dims = led_real_pos['x'].shape
pat_center = [15*mm, 15*mm]
magnification = 10
box_size = [150*um, 150*um]

center_led = [(led_real_pos['x'][dims[0]-1, dims[1]-1] + led_real_pos['x'][0,0])/2, (led_real_pos['y'][0,0] + led_real_pos['y'][dims[0]-1, dims[1]-1])/2]
shift = merge_centers(led_real_pos, pat_center, dims=dims)
shift_led = [-(1/magnification * (led_real_pos['x'] - center_led[0])) + shift[0], -(1/magnification * (led_real_pos['y'] - center_led[1])) + shift[1]]
center_pos = [shift_led[0] + led_real_pos['x'], shift_led[1] + led_real_pos['y']]
center_cb = [-(1/magnification) * pd_real_pos['x'], -(1/magnification) * pd_real_pos['y']]

tmat = sc.linalg.toeplitz(np.mod(np.arange(1,11,1), 2))

for li in range(0, led_real_pos['x'].shape[0]):
    for lj in range(0, led_real_pos['y'].shape[1]):
        border = mask_util.rect_border([center_pos[0][li,lj], center_pos[1][li,lj]], [2.25*mm, 2.25*mm], 250*um)
        gds_mask.add(*border)

        # Adding alignment markers between the spots
        marker_11 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0][li,lj] + led_real_pos['x'][li,lj] - 1.5*mm, shift_led[1][li,lj] + led_real_pos['y'][li,lj] + 1.5*mm])) # top-left
        marker_21 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0][li,lj] + led_real_pos['x'][li,lj] - 1.5*mm, shift_led[1][li,lj] + led_real_pos['y'][li,lj] - 1.5*mm])) # bottom-left
        marker_12 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0][li,lj] + led_real_pos['x'][li,lj] + 1.5*mm, shift_led[1][li,lj] + led_real_pos['y'][li,lj] + 1.5*mm])) # top-right
        marker_22 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0][li,lj] + led_real_pos['x'][li,lj] + 1.5*mm, shift_led[1][li,lj] + led_real_pos['y'][li,lj] - 1.5*mm])) # bottom-right
        gds_mask.add(marker_11, marker_12, marker_21, marker_22)

        center_cb_shift_x, center_cb_shift_y = center_cb[0] + center_pos[0][li,lj], center_cb[1] + center_pos[1][li,lj]
        for idx1 in range(0, center_cb_shift_x.shape[0]):
            for idx2 in range(0, center_cb_shift_x.shape[1]):
                if tmat[idx1, idx2] == 1:
                    rect = gdstk.rectangle(*utils.convert_rect(box_size, [center_cb_shift_x[idx1, idx2], center_cb_shift_y[idx1, idx2]]))
                    gds_mask.add(rect)

In [107]:
# Writing the mask array to the GDS file
gds_array.write_gds('test2.gds')

### Weight Mask

In [16]:
# File parameters 
sheet_height = 205*mm
sheet_width = 280*mm

# Initializing a GDS library
mask_array = gdstk.Library()
masks = mask_array.new_cell("Block Cell")

center = [15*mm, 15*mm]
dither_size = 200
magnification = 10
title = 'Test Weight Mask'

In [18]:
# Creating the cut border 0f 30mm by 20mm around the mask
slide_border = mask_util.rect_border([center[0], center[1]], [30.25*mm, 30.25*mm], 250*um)
masks.add(*slide_border)

# Selecting the type of the mask
sel_cell = copy.deepcopy(net0)

# Smaller block sizes
inner_dims = sel_cell[0,0].shape

# Setting useful values
dims = sel_cell.shape

# Setting the sart, step and end values of the ideal positions
ideal_start_pos, ideal_end_pos, ideal_sep = 1.25*mm, 25*mm, 2.5*mm

# Initializing an empty dictionary to hold ideal positions
pd_ideal_pos = {}
pd_real_pos = {}
led_real_pos = {}

# Generating a meshgrid of values of idela positions
pd_ideal_arr = np.arange(ideal_start_pos, ideal_end_pos, ideal_sep)

# Creating a meshgrid of positions
pd_ideal_pos['x'], pd_ideal_pos['y'] =  np.meshgrid(pd_ideal_arr, pd_ideal_arr)

# Converting the coordinates to GDSTK frame of reference
pd_ideal_pos['x'], pd_ideal_pos['y'] = utils.convert_coords(25*mm, pd_ideal_pos['x'], pd_ideal_pos['y'])

# Shifting the coordinates to the center
pd_ideal_pos['x'], pd_ideal_pos['y'] = utils.shift_to_center(25*mm, pd_ideal_pos['x'], pd_ideal_pos['y'])


# Assigning the created variables
pd_real_pos = copy.deepcopy(PD_dev1_cen)
pd_real_pos['x'], pd_real_pos['y'] = utils.shift_to_center(25*mm, pd_real_pos['x'], pd_real_pos['y'])
led_real_pos = copy.deepcopy(LED_input_cen)

# Finding center position of the matrix
center_led = [(led_real_pos['x'][dims[0]-1, dims[1]-1] + led_real_pos['x'][0,0])/2, (led_real_pos['y'][0,0] + led_real_pos['y'][dims[0]-1, dims[1]-1])/2]

# Finding shifts
shift = utils.merge_centers(led_real_pos, center, dims=dims)

In [19]:
for li in range(0, dims[0]):
    for lj in range(0, dims[1]):

        # Initializing an empty dictionary to hold coordinates and dimensions of the dithered values
        dither_mat = {}
        
        # Diagnostics
        print('li: {} \t, lj: {} \n'.format(li, lj))
        
        # Dithering the values in the 8 by 8 matrix
        dither_mat['y'], dither_mat['x'], dither_mat['dim'] = dit.dither_16x(sel_cell[li,lj].flatten(), ((dither_size/16) * 10**-3)) # Because the output from the dither_16x returns y dimension first
        dither_mat['x'], dither_mat['y'], dither_mat['dim'] = np.array(dither_mat['x'], dtype=object)*mm, np.array(dither_mat['y'], dtype=object)*mm, np.array(dither_mat['dim'], dtype=object)*mm
        
        # Solution for the problem with recasting a matrix of empty arrays. This creates a new matrix of empty arrays of suitable size instead of recasting
        if dither_mat['x'].size == 0:
            empty_mat = np.empty(inner_dims, dtype=object)
            for li_loop1 in range(0, empty_mat.shape[0]):
                for lj_loop1 in range(0, empty_mat.shape[1]):
                    empty_mat[li_loop1,lj_loop1] = np.array([])
            dither_mat['x'], dither_mat['y'], dither_mat['dim'] = empty_mat, empty_mat, empty_mat
        else:
            # Else, do the reshaping
            dither_mat['x'], dither_mat['y'], dither_mat['dim'] = dither_mat['x'].reshape(inner_dims), dither_mat['y'].reshape(inner_dims), dither_mat['dim'].reshape(inner_dims)
        
        # Shifting the positions of the 10 by 10 weight matrices based on the LED positions and the also shifted for all of the patterns to be coincidental on the next layer
        shift_led = [-(1/magnification * (led_real_pos['x'][li,lj] - center_led[0])) + shift[0], -(1/magnification * (led_real_pos['y'][li,lj] - center_led[1])) + shift[1]]
        dither_mat['x'], dither_mat['y'], dither_mat['dim'] = dither_mat['x'] + led_real_pos['x'][li,lj] + shift_led[0], dither_mat['y'] + led_real_pos['y'][li,lj] + shift_led[1], dither_mat['dim']

        # Adding alignment markers between the spots
        marker_11 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0] + led_real_pos['x'][li,lj] - 1.5*mm, shift_led[1] + led_real_pos['y'][li,lj] + 1.5*mm])) # top-left
        marker_21 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0] + led_real_pos['x'][li,lj] - 1.5*mm, shift_led[1] + led_real_pos['y'][li,lj] - 1.5*mm])) # bottom-left
        marker_12 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0] + led_real_pos['x'][li,lj] + 1.5*mm, shift_led[1] + led_real_pos['y'][li,lj] + 1.5*mm])) # top-right
        marker_22 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0] + led_real_pos['x'][li,lj] + 1.5*mm, shift_led[1] + led_real_pos['y'][li,lj] - 1.5*mm])) # bottom-right
        masks.add(marker_11, marker_12, marker_21, marker_22)

        # Looping within the 10 by 10 matrix 
        for idx1 in range(0, dither_mat['x'].shape[0]):
            for idx2 in range(0, dither_mat['x'].shape[1]):
                sel_mat = {}
                sel_mat['x'], sel_mat['y'], sel_mat['dim'] = dither_mat['x'][idx1, idx2], dither_mat['y'][idx1, idx2], dither_mat['dim'][idx1, idx2]
                if sel_mat['x'].size == 0 or sel_mat['y'].size == 0 or type:
                    # Skipping if the matrix is empty. Happens for 8 by 8 row for input
                    continue

                # Shifting the matrix according to the real positions of the photodiodes so the shifts are accounted for
                sel_mat['x'], sel_mat['y'], sel_mat['dim'] = sel_mat['x'] - ((1/magnification) * pd_real_pos['x'][idx1, idx2]), sel_mat['y'] - ((1/magnification) * pd_real_pos['y'][idx1, idx2]), sel_mat['dim']
                
                # Placing rectangles at appropriate positions by looping through the dither blocks
                for idx3 in range(0, sel_mat['x'].shape[0]):
                    pattern = gdstk.rectangle(*utils.convert_rect([sel_mat['dim'][idx3], sel_mat['dim'][idx3]], [sel_mat['x'][idx3], sel_mat['y'][idx3]]))
                    masks.add(pattern)
        
# Adding the description text to the mask
text_pos = (center[0] - 14.5*mm, center[1] - 14.5*mm)
desc = gdstk.text(title, 0.75*mm, text_pos)
masks.add(*desc)

li: 0 	, lj: 0 

li: 0 	, lj: 1 

li: 0 	, lj: 2 

li: 0 	, lj: 3 

li: 0 	, lj: 4 

li: 0 	, lj: 5 

li: 0 	, lj: 6 

li: 0 	, lj: 7 

li: 1 	, lj: 0 

li: 1 	, lj: 1 

li: 1 	, lj: 2 

li: 1 	, lj: 3 

li: 1 	, lj: 4 

li: 1 	, lj: 5 

li: 1 	, lj: 6 

li: 1 	, lj: 7 

li: 2 	, lj: 0 

li: 2 	, lj: 1 

li: 2 	, lj: 2 

li: 2 	, lj: 3 

li: 2 	, lj: 4 

li: 2 	, lj: 5 

li: 2 	, lj: 6 

li: 2 	, lj: 7 

li: 3 	, lj: 0 

li: 3 	, lj: 1 

li: 3 	, lj: 2 

li: 3 	, lj: 3 

li: 3 	, lj: 4 

li: 3 	, lj: 5 

li: 3 	, lj: 6 

li: 3 	, lj: 7 

li: 4 	, lj: 0 

li: 4 	, lj: 1 

li: 4 	, lj: 2 

li: 4 	, lj: 3 

li: 4 	, lj: 4 

li: 4 	, lj: 5 

li: 4 	, lj: 6 

li: 4 	, lj: 7 

li: 5 	, lj: 0 

li: 5 	, lj: 1 

li: 5 	, lj: 2 

li: 5 	, lj: 3 

li: 5 	, lj: 4 

li: 5 	, lj: 5 

li: 5 	, lj: 6 

li: 5 	, lj: 7 

li: 6 	, lj: 0 

li: 6 	, lj: 1 

li: 6 	, lj: 2 

li: 6 	, lj: 3 

li: 6 	, lj: 4 

li: 6 	, lj: 5 

li: 6 	, lj: 6 

li: 6 	, lj: 7 

li: 7 	, lj: 0 

li: 7 	, lj: 1 

li: 7 	, lj: 2

In [68]:
temp_mat = dither_mat['x']
dims

(5, 10)

### Earlier Version

In [30]:
# Creating the cut border 0f 30mm by 20mm around the mask
pat_center = [15*mm, 15*mm]
slide_border = mask_util.rect_border([pat_center[0], pat_center[1]], [30.25*mm, 30.25*mm], 250*um)
masks.add(*slide_border)

# Selecting the type of the mask
sel_cell = copy.deepcopy(net0)

# Smaller block sizes
inner_dims = sel_cell[0,0].shape

# Setting useful values
magnification = 10
dims = sel_cell.shape

In [31]:
# Setting the sart, step and end values of the ideal positions
ideal_start_pos, ideal_end_pos, ideal_sep = 1.25*mm, 25*mm, 2.5*mm

# Initializing an empty dictionary to hold ideal positions
pd_ideal_pos = {}
pd_real_pos = {}
led_real_pos = {}

# Generating a meshgrid of values of idela positions
pd_ideal_arr = np.arange(ideal_start_pos, ideal_end_pos, ideal_sep)
pd_ideal_pos['x'], pd_ideal_pos['y'] =  np.meshgrid(pd_ideal_arr, pd_ideal_arr)
pd_ideal_pos['x'], pd_ideal_pos['y'] = convert_coords(25*mm, pd_ideal_pos['x'], pd_ideal_pos['y'])
pd_ideal_pos['x'], pd_ideal_pos['y'] = shift_to_center(25*mm, pd_ideal_pos['x'], pd_ideal_pos['y'])


# Assigning the created variables
pd_real_pos = copy.deepcopy(PD_dev1_cen)
pd_real_pos['x'], pd_real_pos['y'] = shift_to_center(25*mm, pd_real_pos['x'], pd_real_pos['y'])
led_real_pos = copy.deepcopy(LED_input_cen)

# Inverting the coordinates (Not required since it's been done in the files shared by Alex)
# led_real_pos['x'], led_real_pos['y'] = convert_coords(25*mm, led_real_pos['x'], led_real_pos['y'])
# pd_real_pos['x'], pd_real_pos['y'] = convert_coords(25*mm, pd_real_pos['x'], pd_real_pos['y'])
# pd_real_pos['x'], pd_real_pos['y'] = shift_to_center(25*mm, pd_real_pos['x'], pd_real_pos['y'])

# Finding center position of the matrix
center_led = [(led_real_pos['x'][dims[0]-1, dims[1]-1] + led_real_pos['x'][0,0])/2, (led_real_pos['y'][0,0] + led_real_pos['y'][dims[0]-1, dims[1]-1])/2]

# Finding shifts
shift = utils.merge_centers(led_real_pos, pat_center, dims=dims)

In [32]:
for li in range(0, sel_cell.shape[0]):
    for lj in range(0, sel_cell.shape[1]):
        # Initializing an empty dictionary to hold coordinates and dimensions of the dithered values
        dither_mat = {}

        # Diagnostics
        # print('sum: {}'.format(str(li+lj)))
        # print('li: {} \t lj: {}'.format(str(li), str(lj)))
        
        # Dithering the values in the 8 by 8 matrix
        dither_mat['y'], dither_mat['x'], dither_mat['dim'] = dit.dither_16x(sel_cell[li,lj].flatten(), 0.0125) # Because the output from the dither_16x returns y dimension first
        dither_mat['x'], dither_mat['y'], dither_mat['dim'] = np.array(dither_mat['x'], dtype=object)*mm, np.array(dither_mat['y'], dtype=object)*mm, np.array(dither_mat['dim'], dtype=object)*mm
        
        # Solution for the problem with recasting a matrix of empty arrays. This creates a new matrix of empty arrays of suitable size instead of recasting
        if dither_mat['x'].size == 0:
            # print('empty')
            empty_mat = np.empty(inner_dims, dtype=object)
            for li_loop1 in range(0, empty_mat.shape[0]):
                for lj_loop1 in range(0, empty_mat.shape[1]):
                    empty_mat[li_loop1,lj_loop1] = np.array([])
            dither_mat['x'], dither_mat['y'], dither_mat['dim'] = empty_mat, empty_mat, empty_mat
        else:
            # Else, do the reshaping
            dither_mat['x'], dither_mat['y'], dither_mat['dim'] = dither_mat['x'].reshape(inner_dims), dither_mat['y'].reshape(inner_dims), dither_mat['dim'].reshape(inner_dims)
        
        # Shifting the positions of the 10 by 10 weight matrices based on the LED positions and the also shifted for all of the patterns to be coincidental on the next layer
        shift_led = [-(1/magnification * (led_real_pos['x'][li,lj] - center_led[0])) + shift[0], -(1/magnification * (led_real_pos['y'][li,lj] - center_led[1])) + shift[1]]
        dither_mat['x'], dither_mat['y'], dither_mat['dim'] = dither_mat['x'] + led_real_pos['x'][li,lj] + shift_led[0], dither_mat['y'] + led_real_pos['y'][li,lj] + shift_led[1], dither_mat['dim']
        
        # Debug test by placing a border where the masks are supposed to be
        # border = mask_util.rect_border([shift_led[0] + led_real_pos['x'][li,lj], shift_led[1] + led_real_pos['y'][li,lj]], [2.25*mm, 2.25*mm], 250*um)
        # masks.add(*border)

        # Adding alignment markers between the spots
        marker_11 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0] + led_real_pos['x'][li,lj] - 1.5*mm, shift_led[1] + led_real_pos['y'][li,lj] + 1.5*mm])) # top-left
        marker_21 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0] + led_real_pos['x'][li,lj] - 1.5*mm, shift_led[1] + led_real_pos['y'][li,lj] - 1.5*mm])) # bottom-left
        marker_12 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0] + led_real_pos['x'][li,lj] + 1.5*mm, shift_led[1] + led_real_pos['y'][li,lj] + 1.5*mm])) # top-right
        marker_22 = gdstk.rectangle(*utils.convert_rect([200*um, 200*um], [shift_led[0] + led_real_pos['x'][li,lj] + 1.5*mm, shift_led[1] + led_real_pos['y'][li,lj] - 1.5*mm])) # bottom-right
        masks.add(marker_11, marker_12, marker_21, marker_22)

        # Looping within the 10 by 10 matrix 
        for idx1 in range(0, dither_mat['x'].shape[0]):
            for idx2 in range(0, dither_mat['x'].shape[1]):
                # print('idx1: {} \t idx2: {}'.format(str(idx1), str(idx2)))
                sel_mat = {}
                sel_mat['x'], sel_mat['y'], sel_mat['dim'] = dither_mat['x'][idx1, idx2], dither_mat['y'][idx1, idx2], dither_mat['dim'][idx1, idx2]
                if sel_mat['x'].size == 0 or sel_mat['y'].size == 0:
                    # Skipping if the matrix is empty. Happens for 8 by 8 row for input
                    continue

                # Shifting the matrix according to the real positions of the photodiodes so the shifts are accounted for
                sel_mat['x'], sel_mat['y'], sel_mat['dim'] = sel_mat['x'] - ((1/magnification) * pd_real_pos['x'][idx1, idx2]), sel_mat['y'] - ((1/magnification) * pd_real_pos['y'][idx1, idx2]), sel_mat['dim']
                
                # Placing rectangles at appropriate positions by looping through the dither blocks
                for idx3 in range(0, sel_mat['x'].shape[0]):
                    # print('idx1: {} \t idx2: {} \t idx3: {}'.format(str(idx1), str(idx2), str(idx3)))
                    pattern = gdstk.rectangle(*utils.convert_rect([sel_mat['dim'][idx3], sel_mat['dim'][idx3]], [sel_mat['x'][idx3], sel_mat['y'][idx3]]))
                    masks.add(pattern)

AttributeError: 'float' object has no attribute 'size'

In [67]:
text_pos = (0.5*mm, 0.5*mm)
desc = gdstk.text(r'Input Text', 0.75*mm, text_pos)
masks.add(*desc)

In [68]:
# Writing the mask array to the GDS file
mask_array.write_gds('test.gds')

In [38]:
temp_x = LED_dev1_cen['x']
temp_y = LED_dev1_cen['y']

## Individual Calculations
### Dithering the input values

In [62]:
dither_mat = {}
tmp_mat = dit.dither_16x(sel_cell[0,2].flatten(), ((dither_size/16) * 10**-3))
dither_mat['y'] = np.array(tmp_mat[0], dtype=object).reshape(10,10)*mm
dither_mat['x'] = np.array(tmp_mat[1], dtype=object).reshape(10,10)*mm
dither_mat['dim'] = np.array(tmp_mat[1], dtype=object).reshape(10,10)*mm

In [64]:
dither_mat['y']

array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], dtype=object)

# Archival